# Connect to NYC Filght Database

In [147]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [148]:
client = MongoClient('mongodb://localhost:27017/')
print(client.database_names())

['admin', 'config', 'exhibits', 'flight', 'inventory', 'local', 'orders', 'products', 'sales', 'test-database-1', 'users']


In [149]:
db = client['flight']
db.collection_names()

['flights_nyc', 'flights']

In [150]:
coll = db['flights_nyc']

In [151]:
coll.count()
pprint(coll.find_one())

{'ActualElapsedTime': 128,
 'AirTime': 91,
 'ArrDelay': 0,
 'ArrTime': 1412,
 'CRSArrTime': 1412,
 'CRSDepTime': 1210,
 'CRSElapsedTime': 122,
 'CancellationCode': '',
 'Cancelled': 0,
 'CarrierDelay': 'NA',
 'DayOfWeek': 7,
 'DayofMonth': 20,
 'DepDelay': -6,
 'DepTime': 1204,
 'Dest': 'DTW',
 'Distance': 487,
 'Diverted': 0,
 'FlightNum': 3052,
 'LateAircraftDelay': 'NA',
 'Month': 1,
 'NASDelay': 'NA',
 'Origin': 'EWR',
 'SecurityDelay': 'NA',
 'TailNum': 'N13936',
 'TaxiIn': 10,
 'TaxiOut': 27,
 'UniqueCarrier': 'XE',
 'WeatherDelay': 'NA',
 'Year': 2008,
 '_id': ObjectId('5383814386a8eedea805348f'),
 'aircraft': {'manufacturer': 'EMBRAER', 'model': 'EMB-145EP', 'year': 1997},
 'from': {'city': 'Newark',
          'lat': 40.69249722,
          'long': -74.16866056,
          'state': 'NJ'},
 'to': {'city': 'Detroit',
        'lat': 42.21205889,
        'long': -83.34883583,
        'state': 'MI'}}


### 1. Of the Three Airports, Who Has the Most Flights?

In [8]:
res = coll.aggregate([
    {
        '$group': {
            '_id': {'Airport': '$Origin'},
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    }
])

In [9]:
most_airport = list(res)

In [10]:
most_airport

[{'_id': {'Airport': 'EWR'}, 'count': 175169},
 {'_id': {'Airport': 'LGA'}, 'count': 149686},
 {'_id': {'Airport': 'JFK'}, 'count': 148738}]

### 2. Who Has the Most Cancella*ons and Highest Cancella*on Ra*o?

In [11]:
res = coll.aggregate([
    {
        '$group': {
            '_id': {'Airport': '$Origin'},
            'total_flights': {'$sum': 1},
            'cancelled_flights': {
                '$sum': {
                    '$cond': [{
                        '$eq': ["$Cancelled", 1]
                    }, 1, 0]
                }
            }
        }
    },
    {
        '$project': {
            'ratio': {
                '$divide': ["$cancelled_flights", "$total_flights"]
            }
        }
    },
    {
        '$sort': {'ratio': -1}
    }
])

In [12]:
most_cancellation_airport = list(res)

In [13]:
most_cancellation_airport

[{'_id': {'Airport': 'LGA'}, 'ratio': 0.0513808906644576},
 {'_id': {'Airport': 'EWR'}, 'ratio': 0.03549143969537989},
 {'_id': {'Airport': 'JFK'}, 'ratio': 0.027282873240194166}]

### Tax in /out Times?

In [25]:
res = coll.aggregate([
    {
        '$group': {
            '_id': 1,
            'count': {
                '$sum': {
                    '$cond': [{
                        '$eq': ['$TaxiIn', 'NA'],
                        '$eq': ['$TaxiOut', 'NA']
                    }, 1, 0]
                }
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'count': 1
        }
    }
])

In [26]:
taxin_out_times = list(res)

In [27]:
taxin_out_times

[{'count': 17954}]

In [ ]:
res = coll.aggregate([
    {
        '$group': {
            '_id': {'Airport': '$Origin'},
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    }
])

### Delay by Month
-  Flatten a json 
https://medium.com/@amirziai/flatten-json-on-python-package-index-pypi-9e4951693a5a

In [92]:
res = coll.aggregate([
    {
        '$group': {
            '_id': {
                'Year': '$Year',
                'Month': '$Month'   
            },
            'Avg_ArrDelay': {'$avg': '$ArrDelay'},
            'Avg_DepDelay': {'$avg': '$DepDelay'},
        }
    },
])

In [93]:
delay_by_month = list(res)

In [97]:
dic = delay_by_month

from flatten_json import flatten
dic_flattened = [flatten(d) for d in dic]

import pandas as pd
df = pd.DataFrame(dic_flattened)
df = df.rename(columns={'_id_Year':'Year','_id_Month':'Month','Avg_ArrDelay':'Avg Arrival Delay','Avg_DepDelay':'Avg Departure Delay'})

In [98]:
df['Year_Month'] = df.apply(lambda x: str(int(x.Year)) + '-' + str(int(x.Month)),axis = 1)

In [105]:
df = df.sort_values('Month')

In [106]:
df

,Avg Arrival Delay,Avg Departure Delay,Month,Year,Year_Month
11,6.345677,9.958877,1,2008,2008-1
9,17.417154,16.625239,2,2008,2008-2
8,17.753568,18.062860,3,2008,2008-3
10,13.017217,13.085341,4,2008,2008-4
7,12.062528,12.034898,5,2008,2008-5
6,26.491792,21.279982,6,2008,2008-6
3,20.478830,20.110582,7,2008,2008-7
2,19.260766,19.090104,8,2008,2008-8
5,3.508147,7.551839,9,2008,2008-9
4,0.603974,5.515430,10,2008,2008-10


In [111]:
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime

import plotly
plotly.tools.set_credentials_file(username='awang93', api_key='Ha2fNqLIfAQ00b929raj')

Avg_ArrDelay = go.Scatter(
    x=df.Year_Month,
    y=df['Avg Arrival Delay'],
    name = "Avg Arrival Delay",
    line = dict(color = '#2c2229'),
    opacity = 0.8)

Avg_DepDelay = go.Scatter(
    x=df.Year_Month,
    y=df['Avg Departure Delay'],
    name = "Avg Departure Delay",
    line = dict(color = '#cdc1c5'),
    opacity = 0.8)

data = [Avg_ArrDelay, Avg_DepDelay]

layout = dict(
    title='Time Series with Rangeslider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = dict(data=data,layout=layout)
py.iplot(fig, filename = "line-mode")

### Weather Delays

In [159]:
res = coll.aggregate([
    {
        '$match': {'WeatherDelay': 0}
    },
    {
        '$group': {
            '_id': {
                'Airport': '$Origin',
                'Year': '$Year',
                'Month': '$Month'   
            },
            'Avg_ArrDelay': {'$avg': '$ArrDelay'},
            'Avg_DepDelay': {'$avg': '$DepDelay'},
        }
    },
    {
        '$project': {
            'Avg_ArrDelay': 1,
            'Avg_DepDelay': 1,
            'Avg_Delay': {'$avg': ['$Avg_ArrDelay','$Avg_DepDelay']}
            }
    }
])

In [160]:
weather_delay_by_month = list(res)

In [161]:
weather_delay_by_month[0]

{'_id': {'Airport': 'LGA', 'Year': 2008, 'Month': 12},
 'Avg_ArrDelay': 55.872854077253216,
 'Avg_DepDelay': 44.53057939914163,
 'Avg_Delay': 50.20171673819742}

In [162]:
dic = weather_delay_by_month

from flatten_json import flatten
dic_flattened = [flatten(d) for d in dic]

import pandas as pd
df = pd.DataFrame(dic_flattened)
df = df.rename(columns={'_id_Airport':'Airport','_id_Year':'Year','_id_Month':'Month','Avg_ArrDelay':'Avg Arrival Delay','Avg_DepDelay':'Avg Departure Delay'})

In [164]:
df['Year_Month'] = df.apply(lambda x: str(int(x.Year)) + '-' + str(int(x.Month)),axis = 1)

In [165]:
df = df.sort_values('Month')

In [168]:
df.head()

,Avg Arrival Delay,Avg_Delay,Avg Departure Delay,Airport,Month,Year,Year_Month
35,54.825963,51.111339,47.396715,EWR,1,2008,2008-1
30,49.835984,42.829850,35.823716,LGA,1,2008,2008-1
28,51.572816,45.114298,38.655781,JFK,1,2008,2008-1
33,58.390761,51.237031,44.083302,JFK,2,2008,2008-2
24,54.932770,46.917905,38.903041,LGA,2,2008,2008-2


In [173]:
EWR = df.loc[df['Airport'] == 'EWR']
LGA = df.loc[df['Airport'] == 'LGA']
JFK = df.loc[df['Airport'] == 'JFK']

In [174]:
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime

import plotly
plotly.tools.set_credentials_file(username='awang93', api_key='Ha2fNqLIfAQ00b929raj')

EWR = go.Scatter(
    x=EWR.Year_Month,
    y=EWR['Avg_Delay'] ,
    name = "EWR Airport Avg Delay",
    line = dict(color = '#f2cab9'),
    opacity = 0.8)

LGA = go.Scatter(
    x=LGA.Year_Month,
    y=LGA['Avg_Delay'] ,
    name = "LGA Airport Avg Delay",
    line = dict(color = '#dead00'),
    opacity = 0.8)

JFK = go.Scatter(
    x=JFK.Year_Month,
    y=JFK['Avg_Delay'] ,
    name = "JFK Airport Avg Delay",
    line = dict(color = '#2c2229'),
    opacity = 0.8)

data = [EWR, LGA, JFK]

layout = dict(
    title='Time Series with Rangeslider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = dict(data=data,layout=layout)
py.iplot(fig, filename = "line-mode")

###  Work for Boeing

In [204]:
res = coll.aggregate([
    {
        '$match': {'aircraft.manufacturer': 'BOEING'}
    },
    {
        '$group': {
            '_id': {
                'Destination': '$Dest',
                'Airport': '$Origin'
            },
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    }
])

In [205]:
boeing = list(res)

In [208]:
dic = boeing
from flatten_json import flatten
dic_flattened = [flatten(d) for d in dic]

import pandas as pd
df = pd.DataFrame(dic_flattened)
df = df.rename(columns={'_id_Destination':'Destination','_id_Airport': 'Airport'})

In [209]:
df.head()

,Airport,Destination,count
0,JFK,LAX,9763
1,LGA,ATL,9548
2,EWR,ATL,7189
3,JFK,SFO,6234
4,EWR,IAH,4828


In [210]:
EWR = df.loc[df['Airport'] == 'EWR']
LGA = df.loc[df['Airport'] == 'LGA']
JFK = df.loc[df['Airport'] == 'JFK']

In [211]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=EWR['Destination'],
    y=EWR['count'],
    name='EWR'
)
trace2 = go.Bar(
    x=LGA['Destination'],
    y=LGA['count'],
    name='LGA'
)
trace3 = go.Bar(
    x=JFK['Destination'],
    y=JFK['count'],
    name='JFK'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [231]:
res = coll.aggregate([
    {
        '$match': {'aircraft.manufacturer': 'BOEING'}
    },
    {
        '$group': {
            '_id': {'Destination': '$to.state'},
            'count': {'$sum': 1},
            'lat': {'$avg':'$to.lat'},
            'long': {'$avg':'$to.long'}
        }
    },
    {
        '$sort': {'count': -1}
    }
])

In [232]:
boeing2 = list(res)

In [241]:
boeing2[0]

{'_id': {'Destination': 'CA'},
 'count': 25047,
 'lat': 35.15872090371462,
 'long': -119.71448692674171}

In [244]:
dic = boeing2

from flatten_json import flatten
dic_flattened = [flatten(d) for d in dic]

import pandas as pd
df = pd.DataFrame(dic_flattened)
df = df.rename(columns={'_id_Destination':'name','count':'pop','long':'lon'})

In [245]:
df.head()

,name,pop,lat,lon
0,CA,25047,35.158721,-119.714487
1,FL,24954,27.279049,-81.069200
2,GA,18498,33.639299,-84.424504
3,TX,14334,30.364907,-96.005672
4,IL,8652,41.974560,-87.900510


In [258]:
import plotly.plotly as py
import pandas as pd

df['text'] = 'Beoing Destination: ' + df['name'] + '<br>' + (df['pop']).astype(str)+' times'
limits = [(0,5),(6,10),(11,15),(16,20),(20,35)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","#2c2229"]
cities = []
scale = 10

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = '...',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )